In [6]:
import tensorflow as tf 
from tensorflow.keras import layers, models

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 64     
CHANNELS = 1       
BATCH_SIZE = 32   

train_ds = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\paulo\OneDrive\Área de Trabalho\PDI_CNN\placas',
    validation_split=0.2, 
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\paulo\OneDrive\Área de Trabalho\PDI_CNN\placas',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)
print(f"Classes encontradas: {train_ds.class_names}")
print(f"Total de classes: {len(train_ds.class_names)}")

normalization_layer = layers.Rescaling(1./127.5, offset=-1)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


Found 36576 files belonging to 36 classes.
Using 29261 files for training.
Found 36576 files belonging to 36 classes.
Using 7315 files for validation.
Classes encontradas: ['Sample001', 'Sample002', 'Sample003', 'Sample004', 'Sample005', 'Sample006', 'Sample007', 'Sample008', 'Sample009', 'Sample010', 'Sample011', 'Sample012', 'Sample013', 'Sample014', 'Sample015', 'Sample016', 'Sample017', 'Sample018', 'Sample019', 'Sample020', 'Sample021', 'Sample022', 'Sample023', 'Sample024', 'Sample025', 'Sample026', 'Sample027', 'Sample028', 'Sample029', 'Sample030', 'Sample031', 'Sample032', 'Sample033', 'Sample034', 'Sample035', 'Sample036']
Total de classes: 36


In [ ]:
import tf_keras as keras
import tensorflow_model_optimization as tfmot

def create_esp32_simple_cnn():
    inputs = keras.Input(shape=(64, 64, 1))
    
    x = keras.layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    
    x = keras.layers.SeparableConv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    
    x = keras.layers.SeparableConv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    
    outputs = keras.layers.Dense(36, activation='softmax')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs)

base_model = create_esp32_simple_cnn()

q_aware_model = tfmot.quantization.keras.quantize_model(base_model)

q_aware_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Iniciando Treinamento com QAT...")
q_aware_model.fit(train_ds, validation_data=val_ds, epochs=15) #parei aqui




Iniciando Treinamento com QAT...
Epoch 1/15


915/915 [==============================] - 60s 63ms/step - loss: 3.4610 - accuracy: 0.0556 - val_loss: 3.0677 - val_accuracy: 0.1610
Epoch 2/15
915/915 [==============================] - 25s 27ms/step - loss: 2.8661 - accuracy: 0.1908 - val_loss: 2.4801 - val_accuracy: 0.3911
Epoch 3/15
915/915 [==============================] - 12s 14ms/step - loss: 2.4436 - accuracy: 0.3219 - val_loss: 2.0700 - val_accuracy: 0.5069
Epoch 4/15
915/915 [==============================] - 12s 13ms/step - loss: 2.0915 - accuracy: 0.4358 - val_loss: 1.7276 - val_accuracy: 0.6062
Epoch 5/15
915/915 [==============================] - 16s 17ms/step - loss: 1.8102 - accuracy: 0.5162 - val_loss: 1.4724 - val_accuracy: 0.6652
Epoch 6/15
915/915 [==============================] - 15s 17ms/step - loss: 1.6118 - accuracy: 0.5669 - val_loss: 1.2957 - val_accuracy: 0.7035
Epoch 7/15
915/915 [==============================] - 12s 14ms/step - loss: 1.4583 - accuracy: 0.6

In [ ]:
import numpy as np

def representative_data_gen():
    for input_value, _ in val_ds.take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

with open("modelo_placa_int8.tflite", "wb") as f:
    f.write(tflite_model_quant)

print("Modelo convertido com sucesso para modelo_placa_int8.tflite!")

INFO:tensorflow:Assets written to: C:\Users\paulo\AppData\Local\Temp\tmpjyz2ik46\assets


INFO:tensorflow:Assets written to: C:\Users\paulo\AppData\Local\Temp\tmpjyz2ik46\assets
C:\Users\paulo\AppData\Roaming\Python\Python312\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Modelo convertido com sucesso para modelo_placa_int8.tflite!


In [ ]:
import os

def bin_to_header(filename, var_name='modelo_placa_int8'):
    with open(filename, 'rb') as f:
        data = f.read()
    
    with open(var_name + '.h', 'w') as f:
        f.write(f'unsigned char {var_name}[] = {{\n  ')
        for i, byte in enumerate(data):
            f.write(f'0x{byte:02x}, ')
            if (i + 1) % 12 == 0:
                f.write('\n  ')
        f.write('\n};\n\n')
        f.write(f'unsigned int {var_name}_len = {len(data)};\n')

bin_to_header('modelo_placa_int8.tflite')
print("Arquivo modelo_placa_int8.h gerado com sucesso!")

Arquivo modelo_placa_int8.h gerado com sucesso!
